# Computing the boost factors for Starlight

In [2]:
import sys
import warnings
import matplotlib.pyplot as plt
import pickle
import uproot
import numpy as np
import pandas as pd
# import xgboost as xgb
import sklearn

sys.path.insert(0, '../')
# import analysis_utils as au

# avoid pandas warning
warnings.simplefilter(action='ignore', category=FutureWarning)

import skhep
from skhep import math



In [3]:
help(math.LorentzVector)

Help on class LorentzVector in module skhep.math.vectors:

class LorentzVector(builtins.object)
 |  LorentzVector(x=0.0, y=0.0, z=0.0, t=0.0)
 |  
 |  Class representing a Lorentz vector,
 |  either a 4-dimensional Minkowski space-time vector or a 4-momentum vector.
 |  The 4-vector components can be seen as (x,y,z,t) or (px,py,pz,E).
 |  
 |  Constructors:
 |      __init__(x=0., y=0., z=0., t=0.)
 |      from4vector(avector)
 |      from3vector(vector3d, t)
 |  
 |  Methods defined here:
 |  
 |  __add__(self, other)
 |      Addition with another LorentzVector, i.e. self+other.
 |  
 |  __div__ = __truediv__(self, number)
 |  
 |  __eq__(self, other)
 |      Equality to another LorentzVector, or, equality to zero.
 |      
 |      Example
 |      -------
 |      >>> v1 = ...
 |      >>> v2 = ...
 |      >>> print v1 == v2
 |      >>> print v1 == 0
 |  
 |  __getitem__(self, i)
 |      Get the ith vector component (commencing at 0, of course).
 |  
 |  __iadd__(self, other)
 |      (se

In [12]:
# Beam coordinates     

HalfSqrtSnn   = 2510.
MassOfLead208 = 193.6823
MassOfProton  = 0.93827208816
MomentumBeam  = math.sqrt( HalfSqrtSnn*HalfSqrtSnn*208*208 - MassOfLead208*MassOfLead208 )
MomentumBeamP = math.sqrt( 6500*6500 - MassOfProton*MassOfProton )
    
# pA            = math.LorentzVector( 0, 0,-1, 1 )
# pB            = math.LorentzVector( 0, 0, 1, 1 )
pA            = math.LorentzVector( 0, 0,-MomentumBeam,  HalfSqrtSnn*208 )
pB            = math.LorentzVector( 0, 0, MomentumBeamP, 6500            )


In [13]:
GammaB = 6500./0.93827208816
GammaB

6927.628011131436

In [23]:
GammaA = HalfSqrtSnn*208./193.6823
GammaA

2695.548328370739

In [16]:
pA.boostvector

<Vector3D (x=0.0,y=0.0,z=-0.999999931186163)>

In [17]:
pB.boostvector

<Vector3D (x=0.0,y=0.0,z=0.9999999895816034)>

In [20]:
GammaA2 = 1./math.sqrt(1-0.999999931186163*0.999999931186163)
GammaA2

2695.5483279706914

In [21]:
GammaB2 = 1./math.sqrt(1-0.9999999895816034*0.9999999895816034)
GammaB2

6927.62802791478

### Approach from Aude

In [22]:
from math import *

#############################################
# Calcul de l'energie ds le centre de masse #
#############################################

def getP(m,E):
    return sqrt(E**2-m**2)

def Ecm(m1,m2,E1,E2):
    p1 = getP(m1,E1)
    p2 = getP(m2,E2)
    return sqrt(m1**2+m2**2+2*E1*E2+2*p1*p2)
    
def getEfromCME(Ecm, m1,m2,E1):

	return

#m1 = 0.51099895000e-6  #TeV/c2
#m2 = 21.763563e-6     #TeV/c2
mp = 938.272e-6 	#TeV/c2
mPb = 208*mp
E2 = 4.                 #TeV
E1 = 6.5 #TeV

Ep = E1	# 2016	# TeV
#Ep = E2	# 2013	# TeV

Pp = getP(mp,Ep)
PPb = Pp*82/208
EPb = sqrt(mp**2+ PPb**2)


#print("sqrt(2*E1*E1+2*p1*p1) ", sqrt(2*E1*E1+2*p1*p1))
print("proton momentum ", getP(mp,Ep))
print("center of mass energy in p-p ", Ecm(mp,mp,Ep,Ep))
print("center of mass energy in p-Pb ", Ecm(mp,mPb,Ep,EPb))

############################################################################
# Calcul du transfert d'energie dans la photoproduction de J/Psi exclusive #
############################################################################

def W(E, y):    # E has to be in GeV
    return sqrt(2*E*exp(-y)*MJ)
    
MJ = 3.0969     #GeV

print("For p-Pb")
print("W(E_p = 6.5TeV, y = 4) =", W(6.5e3, 4), "GeV")
print("W(E_p = 6.5TeV, y = 2.5) =", W(6.5e3, 2.5), "GeV")
print("W(E_p = 4TeV, y = 4) = ", W(4e3 , 4), "GeV")
print("W(E_p = 4TeV, y = 2.5) = ", W(4e3, 2.5), "GeV")

print("For Pb-p")
print("W(E_p = 6.5TeV, y = -4.0) =", W(6.5e3, -4.0), "GeV")
print("W(E_p = 6.5TeV, y = -2.5) =", W(6.5e3, -2.4), "GeV")
print("W(E_p = 4TeV, y = -2.5) =", W(4e3 , -2.5), "GeV")
print("W(E_p = 4TeV, y = -4.0) =", W(4e3, -4.0), "GeV")

################################################
# Calcul de la section efficace différentielle #
################################################

BR = 0.05961

def dsigma(N, AEffProd, lum, dy):
    return N/(AEffProd * BR * lum* dy)

# In p-Pb
N= 414  # 2013
N= 28  # incertitude
AEffProd = 0.1845
lum = 3.9e3   #µb   #2013
lum = 8.4e3   #µb   #2016
dy = 1.5

# In Pb-p
N =71   #2013
N=9     #incertitude
N = 915 #2016
N=32
AEffProd = 0.1075948
lum = 4.5e3    #µb  #2013
lum = 12.8e3    #µb  #2016
dy = 1

print("dSigma = ", dsigma(N, AEffProd, lum, dy), " nb-1")


print("gamma (p) = ", Ep/mp)
print("gamma (Pb) = ", EPb/mp)

print("CM energy = ", Ecm(mp,mPb,Ep,EPb))



proton momentum  6.499999932280434
center of mass energy in p-p  13.0
center of mass energy in p-Pb  8.15881969897575
For p-Pb
W(E_p = 6.5TeV, y = 4) = 27.154780922864603 GeV
W(E_p = 6.5TeV, y = 2.5) = 57.48667166481791 GeV
W(E_p = 4TeV, y = 4) =  21.301962740465186 GeV
W(E_p = 4TeV, y = 2.5) =  45.09625547544738 GeV
For Pb-p
W(E_p = 6.5TeV, y = -4.0) = 1482.6008029437248 GeV
W(E_p = 6.5TeV, y = -2.5) = 666.1754829860648 GeV
W(E_p = 4TeV, y = -2.5) = 549.3848599799786 GeV
W(E_p = 4TeV, y = -4.0) = 1163.0477577043666 GeV
dSigma =  0.38978902265015797  nb-1
gamma (p) =  6927.62866205109
gamma (Pb) =  2731.0845310094696
CM energy =  8.15881969897575
